Let's try with a CNN LSTM Neural Network

In [10]:
from numpy import array

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [11]:
n_steps = 4
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

In [12]:
X

array([[[[10],
         [20]],

        [[30],
         [40]]],


       [[[20],
         [30]],

        [[40],
         [50]]],


       [[[30],
         [40]],

        [[50],
         [60]]],


       [[[40],
         [50]],

        [[60],
         [70]]],


       [[[50],
         [60]],

        [[70],
         [80]]]])